In [1]:
import sys
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from nilearn import image
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_design_matrix, plot_stat_map
sys.path.append('..')
from utils.data import Subject, load_participant_list

In [34]:
base_dir = '/Users/hugofluhr/data/LH_dev'
bids_dir = '/Users/hugofluhr/data/LH_dev/fmriprep-23.2.1'
first_level_dir = '/Users/hugofluhr/data/LH_dev/nilearn_first_level'

dev_sub_ids = ['01', '02', '03', '04', '05', '06']
all_sub_ids = load_participant_list(base_dir)

sub_ids = all_sub_ids

In [ ]:
contrast_modulator_RL_paths = [
    glob.glob(os.path.join(first_level_dir, f"sub-{sub_id}_run-test_model_rl*_z_map.nii.gz"))[0]
    for sub_id in sub_ids
]
contrast_modulator_CK_paths = [
    glob.glob(os.path.join(first_level_dir, f"sub-{sub_id}_run-test_model_ck*_z_map.nii.gz"))[0]
    for sub_id in sub_ids
]

contrast_modulator_RL_paths.sort()
contrast_modulator_CK_paths.sort()

assert len(contrast_modulator_RL_paths) == len(sub_ids), "Some subjects are missing RL contrast maps."
assert len(contrast_modulator_CK_paths) == len(sub_ids), "Some subjects are missing CK contrast maps."

print("All subjects are present for both RL and CK.")

In [36]:
contrast_modulator_RL_imgs = [image.load_img(path) for path in contrast_modulator_RL_paths]
contrast_modulator_CK_imgs = [image.load_img(path) for path in contrast_modulator_CK_paths]

# Effect of RL values only

In [37]:
X_RL = pd.DataFrame([1] * len(contrast_modulator_RL_imgs), columns=['RL_value'])

In [52]:
second_level_model_RL = SecondLevelModel(mask_img=bartra_mask_path)
second_level_model_RL = second_level_model_RL.fit(contrast_modulator_RL_imgs, design_matrix=X_RL)

In [ ]:
z_map_RL = second_level_model_RL.compute_contrast(output_type='z_score')

In [ ]:
plot_stat_map(z_map_RL, threshold=1, display_mode='mosaic', title='Average effect of RL parametric modulator')


# Effect of CK values only

In [43]:
X_CK = pd.DataFrame([1] * len(contrast_modulator_CK_imgs), columns=['intercept'])

In [44]:
second_level_model_CK = SecondLevelModel()#mask_img=bartra_mask_path)
second_level_model_CK = second_level_model_CK.fit(contrast_modulator_CK_imgs, design_matrix=X_CK)

In [45]:
#z_map_CK = second_level_model_CK.compute_contrast(second_level_contrast='intercept', output_type='z_score')
z_map_CK = second_level_model_CK.compute_contrast(output_type='z_score')

In [46]:
from nilearn.glm import threshold_stats_img

thresholded_map1, threshold1 = threshold_stats_img(
    z_map_RL,
    alpha=0.001,
    height_control="fpr",
    cluster_threshold=10,
    two_sided=True,
)

In [ ]:
threshold1

In [ ]:
plot_stat_map(z_map_CK, threshold=2., display_mode='mosaic', title='Average effect of CK parametric modulator')
